In [28]:
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import numpy as np

In [4]:
env = gym.make('FrozenLake-v0')
# env = gym.make('')

[2017-04-24 19:13:21,779] Making new env: FrozenLake-v0


### Q-Learning

- Initialize $V(s)$ arbitrarily
- Repeat for each episode
- Initialize s
- Repeat (for each step of episode)
- -    $\alpha \leftarrow$ action given by $\pi$ for $s$
- - Take action a, observe reward r, and next state s'
- - $V(s) \leftarrow V(s) + \alpha [r = \gamma V(s') - V(s)]$ 
- - $s \leftarrow s'$
- until $s$ is terminal

In [16]:
#Initialize table with all zeros
Q = np.zeros([ env.observation_space.n, env.action_space.n])
# Set learning parameters
lr = .85
y = .99
num_episodes = 2000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    # The Q-Table learning algorithm
    while j < 99:
        j+=1
        
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)

In [13]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[  5.16194512e-01   4.56917349e-03   3.99235349e-03   8.73599358e-03]
 [  2.34689427e-03   3.17131271e-03   1.84516836e-03   3.80200903e-01]
 [  0.00000000e+00   5.73162791e-03   3.59071589e-03   3.73430729e-01]
 [  3.72211594e-04   7.27144109e-04   1.96668156e-03   3.35339450e-01]
 [  3.88179114e-01   4.76648759e-04   2.51088689e-04   1.68876131e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.27647717e-05   3.17437559e-04   9.04053753e-06   1.06373197e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.73311393e-03   6.36284432e-05   1.81904768e-03   2.42404334e-01]
 [  5.65852667e-05   2.47215255e-01   3.27333564e-04   1.26995321e-04]
 [  5.70085312e-02   4.21881319e-04   3.98522005e-04   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   6.34876112e-04   1.74247579e-01   6

In [7]:
print("Score over time: " +  str(sum(rList)/num_episodes))

Score over time: 0.6585


In [19]:
cd ..

/Users/miguel/Jottacloud/HedgingRL


In [112]:
import gym
import gym.envs
import numpy as np

In [37]:
import gym_bs

In [148]:
gym.spaces.Tuple?

In [149]:
env = gym.make('bs-v0')

[2017-04-25 17:59:30,515] Making new env: bs-v0


In [150]:
env.observation_space
# env.action_space.contains(1)

Tuple(Box(1,), Box(1,), Box(1,))

In [100]:
True if 2>=3 else False

False

In [97]:
env.action_space.sample()

1

In [162]:
env.observation_space.sample()

(array([ 4.43761995]), array([ 0.49800923]), array([-98.87277771]))

In [115]:
# Q = np.zeros([ env.observation_space.n, env.action_space.n])
# Set learning parameters
lr = .85
y = .99
num_episodes = 2000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    
    done = False
    while not done:
        a = env.action_space.sample()
        s1, r, done, _ = env.step(a)
        

In [134]:
from blackscholes import blackScholes
blackScholes(0, 1, 1, 0.1)

/Users/miguel/Jottacloud/HedgingRL/blackscholes/mathfin.py:40: RuntimeWarning: invalid value encountered in double_scalars
  d1 = np.log(S / K) / sigma / sqrt_tau + 0.5 * sigma_sqrt_tau


{'delta': nan, 'gamma': nan, 'npv': 0.0, 'theta': nan, 'vega': nan}

In [151]:
K 

NameError: name 'K' is not defined